In [1]:
import torch, os, tqdm
import numpy as np
import pandas as pd
import albumentations as A
import torch.nn.functional as F


from data import dataset
from models import Architecture
from utils import encoder, save_model
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

In [4]:
ocr_transformer = A.Compose([
    A.Resize(width=224, height=50),
    # A.Normalize(mean=[133.98177105, 131.71614195, 142.1675694], std=[11.89496205, 12.33475545, 11.7366861]),
    A.OneOf([
        A.MotionBlur(p=.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0525, scale_limit=0.1, rotate_limit=1, p=1),
    A.GridDistortion(p=.2, border_mode=1),
    A.OneOf([
        A.IAASharpen(),
        A.IAAEmboss(),
        A.RandomBrightnessContrast(p=0.2),
    ]),
    ToTensorV2(),
])

val_transformer = A.Compose([
    A.Resize(width=224, height=50),
    # A.Normalize(mean=[133.98177105, 131.71614195, 142.1675694], std=[11.89496205, 12.33475545, 11.7366861]),
    ToTensorV2(),
])

C:\Users\yerda\anaconda3\lib\site-packages\albumentations\imgaug\transforms.py:231: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
C:\Users\yerda\anaconda3\lib\site-packages\albumentations\imgaug\transforms.py:174: FutureWarning: This augmentation is deprecated. Please use Emboss instead
  warnings.warn("This augmentation is deprecated. Please use Emboss instead", FutureWarning)


In [5]:
val = pd.read_csv("dataset/val_annotation.csv")
test = pd.read_csv("dataset/test_annotation.csv")
train = pd.read_csv("dataset/train_annotation.csv")
train.drop(8, inplace=True)
train.drop(64, inplace=True)
train.drop(117, inplace=True)
train.drop(213, inplace=True)

In [11]:
trainLoader = DataLoader(dataset(train, encoder, max_sequence=19, transformer=ocr_transformer), batch_size=64, shuffle=True)
valLoader = DataLoader(dataset(val, encoder, max_sequence=19, transformer=val_transformer), batch_size=64, shuffle=False)
testLoader = DataLoader(dataset(test, encoder, max_sequence=19, transformer=val_transformer), batch_size=64, shuffle=False)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model = Architecture(num_classes=37).to(device)
loss_fn = torch.nn.CTCLoss(blank=0, reduction="mean", zero_infinity=False)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

In [9]:
os.makedirs("models", exist_ok=True)

In [12]:
num_epochs, prev_loss = 10000, torch.inf
train_loss, val_loss = [], []

for epoch in tqdm.trange(num_epochs):
    model.train()
    loss_list, count = 0, 0
    for images, targets, input_lengths, target_lengths in trainLoader:
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        predictions = model(images)
        predictions = F.log_softmax(predictions, dim=2)
        loss = loss_fn(predictions.permute(1, 0, 2), targets, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()
        loss_list += loss.cpu().item()
        count += 1
    loss = loss_list/count
    train_loss.append(loss)
    
    model.eval()
    @torch.no_grad()
    def validation():
        val_loss, val_counter = 0, 0
        for images, targets, input_lengths, target_lengths in valLoader:
            images = images.to(device)
            targets = targets.to(device)
            predictions = model(images)
            predictions = F.log_softmax(predictions, dim=2)
            loss = loss_fn(predictions.permute(1, 0, 2), targets, input_lengths, target_lengths)
            val_loss += loss.cpu().item()
            val_counter += 1
        return val_loss / val_counter
    v_loss = validation()
    val_loss.append(v_loss)
    prev_loss = save_model("model", epoch, prev_loss, v_loss, model, optimizer)
    
    print(f"Epoch: {epoch} | Training loss {loss} | Validation loss {v_loss}")

  0%|                                                                             | 1/10000 [00:19<55:18:06, 19.91s/it]

The best model was saved!
Epoch: 0 | Training loss 2.38371255248785 | Validation loss 2.0298477113246918



  0%|                                                                             | 2/10000 [00:35<48:41:15, 17.53s/it]

The best model was saved!
Epoch: 1 | Training loss 1.8886383771896362 | Validation loss 1.6351169049739838



  0%|                                                                             | 3/10000 [00:50<44:33:16, 16.04s/it]

...

Epoch: 435 | Training loss 0.47838663309812546 | Validation loss 1.7277981042861938



  4%|███▏                                                                     | 437/10000 [1:58:23<39:13:07, 14.76s/it]

Epoch: 436 | Training loss 0.48517264518886805 | Validation loss 1.7354248762130737


  4%|███▏                                                                     | 437/10000 [1:58:24<43:11:03, 16.26s/it]


KeyboardInterrupt: 

In [14]:
# pd.DataFrame({
#     "epoch": range(0, len(val_loss)),
#     "train_loss": train_loss[:],
#     "val_loss": val_loss,
# }).to_csv("lossesHackathon.csv", index=False)

In [13]:
np.array([0.04664691, 0.04837159, 0.04602622]) * 255

0.0001